In [5]:

import os
import warnings
from argparse import ArgumentParser
import matplotlib.patches as patches
import torch
from tqdm import tqdm
import cv2
from torch.utils import data
# 개별 json 라벨 파일을 이용해 학습 데이터 리스트 생성
from glob import glob
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os
from nets import nn
from utils import util
from utils.dataset import Dataset
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
import random
import openslide
import copy
import random
from time import time

import math
import numpy
import torch
import torchvision
from torch.nn.functional import cross_entropy
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
params={'names':{
  0: 'pd-l1 negative tumor cell',
  1: 'pd-l1 positive tumor cell',
  2: 'non-tumor cell'}}

# params={'names':{
#   0: 'pd-l1 negative tumor cell',
#   1: 'pd-l1 positive tumor cell'}}

device: cuda:0


In [6]:
save_dir='../../model/yolov11/'
model = nn.yolo_v11_m(len(params['names'])).to(device)
checkpoint_path = os.path.join(save_dir, 'best_model.pt')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    
def wh2xy(x):
    y = x.clone() if isinstance(x, torch.Tensor) else numpy.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y
   
def non_max_suppression(outputs, confidence_threshold=0.001, iou_threshold=0.65, class_thresholds=None):
    """
    빠른 클래스별 NMS - 성능 최적화 버전
    """
    max_wh = 7680
    max_det = 300
    max_nms = 30000

    bs = outputs.shape[0]
    nc = outputs.shape[1] - 4
    
    # 빠른 필터링을 위해 가장 낮은 threshold 사용
    min_conf = confidence_threshold
    if class_thresholds:
        min_conf = min(min(class_thresholds.values()), confidence_threshold)
    
    # 전체 confidence가 낮은 것들 먼저 제거
    xc = outputs[:, 4:4 + nc].amax(1) > min_conf
    
    output = [torch.zeros((0, 6), device=outputs.device)] * bs
    
    for xi, x in enumerate(outputs):  # image index, image inference
        x = x.transpose(0, -1)[xc[xi]]
        
        if not x.shape[0]:
            continue

        # 박스와 클래스 분리
        box, cls = x.split((4, nc), 1)
        box = wh2xy(box)
        
        # 각 검출의 최고 클래스와 confidence 찾기
        conf, j = cls.max(1, keepdim=True)
        x = torch.cat((box, conf, j.float()), 1)
        
        # 클래스별 threshold 적용 (간단한 방식)
        if class_thresholds:
            keep = torch.zeros(x.shape[0], dtype=torch.bool, device=x.device)
            for i, detection in enumerate(x):
                class_id = int(detection[5].item())
                threshold = class_thresholds.get(class_id, confidence_threshold)
                if detection[4].item() >= threshold:
                    keep[i] = True
            x = x[keep]
        else:
            x = x[x[:, 4] > confidence_threshold]
        
        if not x.shape[0]:
            continue
            
        # confidence로 정렬하고 상위 max_nms개만 유지
        x = x[x[:, 4].argsort(descending=True)[:max_nms]]
        
        # 빠른 NMS - PyTorch 내장 함수 사용
        c = x[:, 5:6] * max_wh  # 클래스별 offset
        boxes = x[:, :4] + c
        scores = x[:, 4]
        
        # NMS 적용
        keep = torchvision.ops.nms(boxes, scores, iou_threshold)
        if keep.shape[0] > max_det:
            keep = keep[:max_det]
        
        output[xi] = x[keep]
    
    return output
    
def pred_patch(torch_patch, model, start_x, start_y, magnification):
    model.eval()
    
    # 클래스별 개별 confidence threshold 설정
    class_thresholds = {
        0: 0.4,  # pd-l1 negative tumor cell
        1: 0.5,  # pd-l1 positive tumor cell 
        2: 0.3   # non-tumor cell
    }
    
    negative_tumor = []
    positive_tumor = []
    non_tumor = []
    
    with torch.no_grad():
        with torch.amp.autocast('cuda'):
            pred = model(torch_patch)
        
        # 빠른 NMS 적용
        results = non_max_suppression(pred, confidence_threshold=0.3, 
                                    iou_threshold=0.5, class_thresholds=class_thresholds)
        
        if len(results[0]) > 0:
            # 벡터화된 처리로 속도 향상
            detections = results[0]
            xyxy = detections[:, :4]
            confs = detections[:, 4]
            cls_ids = detections[:, 5]
            
            # 중심점 계산 (벡터화)
            centers_x = (xyxy[:, 0] + xyxy[:, 2]) / 2
            centers_y = (xyxy[:, 1] + xyxy[:, 3]) / 2
            
            # 실제 좌표 계산
            actual_x = start_x + centers_x * magnification
            actual_y = start_y + centers_y * magnification
            
            # 클래스별로 분리 (벡터화)
            for i in range(len(detections)):
                cls_id = int(cls_ids[i].item())
                cell_data = {
                    'x': actual_x[i].item(), 
                    'y': actual_y[i].item(), 
                    'cls_id': cls_id,
                    'confidence': confs[i].item()
                }
                
                if cls_id == 0:
                    negative_tumor.append(cell_data)
                elif cls_id == 1:
                    positive_tumor.append(cell_data)
                else:
                    non_tumor.append(cell_data)
    
    return negative_tumor, positive_tumor, non_tumor

In [7]:
def create_asap_xml(negative_tumor, positive_tumor, non_tumor, output_path):
    """
    세포 검출 결과를 ASAP XML 형식으로 저장하는 함수
    
    Args:
        negative_tumor: PD-L1 negative tumor cells 리스트
        positive_tumor: PD-L1 positive tumor cells 리스트  
        non_tumor: Non-tumor cells 리스트
        output_path: 저장할 XML 파일 경로
    """
    
    # 루트 엘리먼트 생성
    root = ET.Element("ASAP_Annotations")
    
    # Annotations 엘리먼트 생성
    annotations = ET.SubElement(root, "Annotations")
    
    annotation_id = 0
    
    # Negative tumor cells 추가 (빨간색)
    for cell in negative_tumor:
        annotation = ET.SubElement(annotations, "Annotation")
        annotation.set("Name", f"Annotation {annotation_id}")
        annotation.set("Type", "Dot")
        annotation.set("PartOfGroup", "Negative Tumor")
        annotation.set("Color", "#0000FF")  # 빨간색
        
        coordinates = ET.SubElement(annotation, "Coordinates")
        coordinate = ET.SubElement(coordinates, "Coordinate")
        coordinate.set("Order", "0")
        coordinate.set("X", str(float(cell['x'])))
        coordinate.set("Y", str(float(cell['y'])))
        
        annotation_id += 1
    
    # Positive tumor cells 추가 (파란색)
    for cell in positive_tumor:
        annotation = ET.SubElement(annotations, "Annotation")
        annotation.set("Name", f"Annotation {annotation_id}")
        annotation.set("Type", "Dot")
        annotation.set("PartOfGroup", "Positive Tumor")
        annotation.set("Color", "#FF0000")  # 파란색
        
        coordinates = ET.SubElement(annotation, "Coordinates")
        coordinate = ET.SubElement(coordinates, "Coordinate")
        coordinate.set("Order", "0")
        coordinate.set("X", str(float(cell['x'])))
        coordinate.set("Y", str(float(cell['y'])))
        
        annotation_id += 1
    
    # Non-tumor cells 추가 (녹색)
    # for cell in non_tumor:
    #     annotation = ET.SubElement(annotations, "Annotation")
    #     annotation.set("Name", f"Annotation {annotation_id}")
    #     annotation.set("Type", "Dot")
    #     annotation.set("PartOfGroup", "Non Tumor")
    #     annotation.set("Color", "#00FF00")  # 녹색
        
    #     coordinates = ET.SubElement(annotation, "Coordinates")
    #     coordinate = ET.SubElement(coordinates, "Coordinate")
    #     coordinate.set("Order", "0")
    #     coordinate.set("X", str(float(cell['x'])))
    #     coordinate.set("Y", str(float(cell['y'])))
        
    #     annotation_id += 1
    
    # AnnotationGroups 엘리먼트 생성
    annotation_groups = ET.SubElement(root, "AnnotationGroups")
    
    # Negative Tumor 그룹
    group1 = ET.SubElement(annotation_groups, "Group")
    group1.set("Name", "Negative Tumor")
    group1.set("PartOfGroup", "None")
    group1.set("Color", "#0000FF")
    attributes1 = ET.SubElement(group1, "Attributes")
    
    # Positive Tumor 그룹
    group2 = ET.SubElement(annotation_groups, "Group")
    group2.set("Name", "Positive Tumor")
    group2.set("PartOfGroup", "None")
    group2.set("Color", "#FF0000")
    attributes2 = ET.SubElement(group2, "Attributes")
    
    # Non Tumor 그룹
    # group3 = ET.SubElement(annotation_groups, "Group")
    # group3.set("Name", "Non Tumor")
    # group3.set("PartOfGroup", "None")
    # group3.set("Color", "#00FF00")
    # attributes3 = ET.SubElement(group3, "Attributes")
    
    # XML을 예쁘게 포맷팅하여 저장
    rough_string = ET.tostring(root, 'unicode')
    reparsed = minidom.parseString(rough_string)
    pretty_xml = reparsed.toprettyxml(indent="	")
    
    # <?xml version... 라인을 원하는 형태로 수정
    lines = pretty_xml.split('\n')
    lines[0] = '<?xml version="1.0"?>'
    pretty_xml = '\n'.join(lines[1:])  # 빈 라인 제거
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(pretty_xml)
    
    # print(f"XML 파일이 저장되었습니다: {output_path}")
    # print(f"총 검출된 세포 수:")
    # print(f"  - Negative tumor cells: {len(negative_tumor)}개")
    # print(f"  - Positive tumor cells: {len(positive_tumor)}개") 
    # print(f"  - Non-tumor cells: {len(non_tumor)}개")
    # print(f"  - 전체: {len(negative_tumor) + len(positive_tumor) + len(non_tumor)}개")



In [8]:
import pyvips
slide_path=glob('../../data/WSI_test/*.ndpi')
image_size=512 # 모델 입력 크기
origin_mpp=0.25
output_mpp=0.5
original_size=int(image_size*output_mpp/origin_mpp) #1122
magnification=original_size/image_size
count=0
for i in tqdm(range(len(slide_path))):

    file_name=os.path.basename(slide_path[i]).split('.')[0]
    slide=openslide.OpenSlide(slide_path[i])
    thumbnail=slide.get_thumbnail((slide.dimensions[0]//64, slide.dimensions[1]//64))
    slide = pyvips.Image.new_from_file(slide_path[i])

    thumb_mask=cv2.threshold(255-np.array(thumbnail.convert('L')),30,255,cv2.THRESH_BINARY)[1]
    thumb_mask=cv2.morphologyEx(thumb_mask,cv2.MORPH_CLOSE,np.ones((15,15),np.uint8))
    thumb_mask=cv2.morphologyEx(thumb_mask,cv2.MORPH_OPEN,np.ones((5,5),np.uint8))
    negative_tumor=[]
    positive_tumor=[]
    non_tumor=[]
    for patch_row in range(slide.width//image_size-1):
        for patch_col in range(slide.height//image_size-1):
            if np.sum(thumb_mask[(patch_col*image_size)//64:((patch_col+1)*image_size)//64,(patch_row*image_size)//64:((patch_row+1)*image_size)//64])>0:
                count+=1
                patch=slide.crop(patch_row*image_size, patch_col*image_size, image_size, image_size)
                patch=np.ndarray(buffer=patch.write_to_memory(),
                            dtype=np.uint8,
                            shape=[patch.height, patch.width, patch.bands])
                torch_patch=torch.from_numpy(np.array(patch)[:,:,:3]).permute(2,0,1).unsqueeze(0).float()/255.
                torch_patch=torch_patch.to(device)
                temp_negative_tumor, temp_positive_tumor, temp_non_tumor = pred_patch(torch_patch, model, patch_row*image_size, patch_col*image_size, 1)
                
                # 메모리 효율적인 extend - 빈 리스트 체크로 불필요한 연산 방지
                if temp_negative_tumor:
                    negative_tumor.extend(temp_negative_tumor)
                if temp_positive_tumor:
                    positive_tumor.extend(temp_positive_tumor)
                # if temp_non_tumor:
                #     non_tumor.extend(temp_non_tumor)
    # XML 파일 생성
    output_xml_path = f"../../results/WSI_IHC_nuclei_detection/{file_name}.xml"
    os.makedirs("../../results/WSI_IHC_nuclei_detection", exist_ok=True)

    create_asap_xml(negative_tumor, positive_tumor, non_tumor, output_xml_path)

100%|██████████| 5/5 [20:12<00:00, 242.47s/it]
